In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter # module for using tensorboard
from torch.autograd import Variable

In [2]:
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [3]:
# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
writer = SummaryWriter() # tensorboard writer

In [6]:
# Training the Model
global_step = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        global_step += 1
        
        if (i+1) % 100 == 0:
            writer.add_scalar("loss", loss.data[0], global_step+1)
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch: [1/5], Step: [100/600], Loss: 1.0538
Epoch: [1/5], Step: [200/600], Loss: 0.8892
Epoch: [1/5], Step: [300/600], Loss: 1.0069
Epoch: [1/5], Step: [400/600], Loss: 1.0160
Epoch: [1/5], Step: [500/600], Loss: 0.8567
Epoch: [1/5], Step: [600/600], Loss: 1.0047
Epoch: [2/5], Step: [100/600], Loss: 0.8343
Epoch: [2/5], Step: [200/600], Loss: 0.9019
Epoch: [2/5], Step: [300/600], Loss: 0.9002
Epoch: [2/5], Step: [400/600], Loss: 0.8481
Epoch: [2/5], Step: [500/600], Loss: 0.8698
Epoch: [2/5], Step: [600/600], Loss: 0.7914
Epoch: [3/5], Step: [100/600], Loss: 0.8406
Epoch: [3/5], Step: [200/600], Loss: 0.7636
Epoch: [3/5], Step: [300/600], Loss: 0.8300
Epoch: [3/5], Step: [400/600], Loss: 0.8250
Epoch: [3/5], Step: [500/600], Loss: 0.8774
Epoch: [3/5], Step: [600/600], Loss: 0.7355
Epoch: [4/5], Step: [100/600], Loss: 0.8019
Epoch: [4/5], Step: [200/600], Loss: 0.6608
Epoch: [4/5], Step: [300/600], Loss: 0.9006
Epoch: [4/5], Step: [400/600], Loss: 0.6849
Epoch: [4/5], Step: [500/600], L

In [5]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(model.state_dict(), 'model.pth')

Accuracy of the model on the 10000 test images: 83 %
